<a href="https://colab.research.google.com/github/r-prateek/Indian-Currency-Notes-Classifier/blob/main/Notes_Classifier_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Kaggle Setup**


*   To import the dataset directly from Kaggle
*   To unzip and store the dataset in Colab




In [2]:
! pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 4.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp37-none-any.whl size=73269 sha256=0b21657fa74d2180de4192a86d6aa76fc47456a41d6c71b33c6c37e2a440aa2f
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rprateek","key":"2200b4d69cd1cf774e7889f25cde8f8f"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [7]:
! kaggle datasets download -d vishalmane109/indian-currency-note-images-dataset-2020

 99% 2.30G/2.31G [00:30<00:00, 45.4MB/s]
100% 2.31G/2.31G [00:30<00:00, 80.5MB/s]


In [ ]:
! unzip indian-currency-note-images-dataset-2020.zip -d dataset

#**Setting up Prerequisites for the Model**

Import libraries required




In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

Set up the location of the training and testing dataset

In [10]:
training_path = "/content/dataset/Indian currency dataset v1/training"
test_path = "/content/dataset/Indian currency dataset v1/validation"

**Improve dataset through data augmentation**

*  Avoids overfitting
*  Helps in increasing accuracy

In [11]:
dataGen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=5,
    horizontal_flip=True,
    vertical_flip=True,
)

In [12]:
train_dataset_aug = dataGen.flow_from_directory(
    training_path,
    target_size = (256,256),
    batch_size = 32,
    shuffle=True,
    seed = 865
)

Found 3566 images belonging to 8 classes.


In [13]:
test_dataset_aug = dataGen.flow_from_directory(
    test_path,
    target_size = (256,256),
    batch_size = 32,
    shuffle=True,
    seed = 865
)

Found 345 images belonging to 8 classes.


#**Creating the CNN Model**

In [14]:
model = tf.keras.models.Sequential()

# Feature extraction part
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))

# Neural network - For classification
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='softmax'))

#Compile
model.compile(optimizer = 'adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics = ['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        3

In [1]:
history = model.fit(train_dataset_aug, validation_data=test_dataset_aug, epochs = 15, batch_size=20)

NameError: ignored